**AI_Assignment (Vijayi_WFH_Tech_internship)**

**Name -** Anil Kushwaha

**Mob.no -**  9829159839

**Email -** kumaranil48309@gmail.com

**PART A:** Task: Extract and Categorize Tasks from Unannotated

In [ ]:
import nltk  # NLTK for text processing
import re  # Regular expressions for text manipulation
from nltk.corpus import stopwords  # Stopwords corpus from NLTK
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer  # Tokenizers for sentences and words
from nltk.tag import pos_tag  # Part-of-speech tagging from NLTK
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDF feature extraction
from sklearn.cluster import KMeans  # KMeans clustering algorithm
import numpy as np  # Numpy for numerical operations
import pandas as pd  # Pandas for data manipulation
from sklearn.model_selection import train_test_split  # Train-test split for model evaluation
from sklearn.linear_model import LogisticRegression  # Logistic Regression for classification
from sklearn.metrics import accuracy_score, precision_score, recall_score  # Model performance metrics


try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Function to preprocess the text by removing punctuation, converting to lowercase, and removing stopwords
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    stop_words = set(stopwords.words('english'))
    words = [word for word in word_tokenize(text) if word not in stop_words]
    return " ".join(words)

# Function to identify task-related sentences from the input text
def identify_tasks(text):
    tokenizer = PunktSentenceTokenizer(nltk.data.find('tokenizers/punkt'))
    sentences = tokenizer.tokenize(text)
    tasks = []
    for sentence in sentences:
        if sentence.strip() == "":
            continue
        words = word_tokenize(sentence)
        tagged_words = pos_tag(words)

        # Identify tasks based on sentence structure and keywords
        if tagged_words and tagged_words[0][1].startswith('VB'):
            tasks.append(sentence)
        elif re.search(r'\b(by|deadline|until)\b', sentence, re.IGNORECASE):
            tasks.append(sentence)
        elif any(tag.startswith('NNP') for _, tag in tagged_words):
            tasks.append(sentence)
    return tasks

# Function to categorize identified tasks using KMeans clustering
def categorize_tasks(tasks):
    if not tasks:
        return {}

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(tasks)

    n_clusters = min(3, len(tasks))
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    kmeans.fit(X)
    labels = kmeans.labels_

    categories = {}
    for i, task in enumerate(tasks):
        category_index = labels[i]
        if category_index not in categories:
            categories[category_index] = []
        categories[category_index].append(task)

    # Assign category names based on representative tasks
    category_names = {}
    for category_index in categories:
        representative_index = np.argmin(kmeans.transform(X[categories[category_index]].toarray()).sum(axis=1))
        representative_task = tasks[categories[category_index][representative_index]]
        category_names[category_index] = representative_task.split()[0].capitalize()

    categorized_tasks = {}
    for category_index, task_list in categories.items():
        categorized_tasks[category_names[category_index]] = task_list
    return categorized_tasks

# Function to extract who is responsible and the deadline from a given task sentence
def extract_who_when(task):
    who = None
    when = None
    match = re.search(r'(.*) (should|must|has to) (.*) (by|at|on) (.*)', task, re.IGNORECASE)
    if match:
        who = match.group(1).strip()
        when = match.group(5).strip()
    elif match := re.search(r'(.*) (should|must|has to) (.*)', task, re.IGNORECASE):
        who = match.group(1).strip()
    return who, when

# Function to process text, extract tasks, categorize them, and retrieve responsible persons and deadlines
def process_text_and_extract_tasks(text):
    preprocessed_text = preprocess_text(text)
    tasks = identify_tasks(preprocessed_text)
    categorized_tasks = categorize_tasks(tasks)

    extracted_tasks_info = []

    for category, task_list in categorized_tasks.items():
        for task in task_list:
            who, when = extract_who_when(task)
            extracted_tasks_info.append({"task": task, "category": category, "who": who, "when": when})

    return extracted_tasks_info

# Example input text for task extraction
text = """Rahul wakes up early every day. He goes to college in the morning and comes back at 3 pm.
At present, Rahul is outside. He has to buy the snacks for all of us. John should clean the room by 5 pm today.
Maria must submit the report. Team should review the proposal. The client has to approve the budget."""

extracted_info = process_text_and_extract_tasks(text)

# Print extracted tasks with category, responsible person, and deadline
for task_info in extracted_info:
    print(task_info)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


................................................................................................................................................................

**PART B:** Customer Review Classification

Build a machine learning model to classify customer reviews as positive or negative. This is a basic text classification problem.

The candidate will demonstrate skills in text preprocessing, feature extraction, and training a simple ML model. (30 marks)

In [ ]:
import pandas as pd  # Pandas for data manipulation
import numpy as np  # Numpy for numerical operations
import nltk  # NLTK for text processing
from nltk.corpus import stopwords  # Stopwords corpus from NLTK
from nltk.tokenize import word_tokenize  # Word tokenization from NLTK
import re  # Regular expressions for text manipulation
from sklearn.model_selection import train_test_split  # Train-test split for model evaluation
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDF feature extraction
from sklearn.linear_model import LogisticRegression  # Logistic Regression for classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # Model performance metrics
import joblib  # For saving and loading models

from google.colab import files  # Import the files module
print("Please upload files :")  # amazon_reviews.csv
uploaded = files.upload()

# 1. Data Collection (Amazon Reviews)
try:
    df = pd.read_csv("amazon_reviews.csv")  # Replace with your actual path
except FileNotFoundError:
    print("Error: amazon_reviews.csv not found. Please download and place it in the correct directory.")
    exit()

# Assuming your Amazon dataset has columns like 'reviewText' and 'overall' or 'review' and 'rating'
# Adapt these column names if your dataset is different
df.rename(columns={"reviewText": "review", "overall": "rating"}, inplace=True)
df = df[['review', 'rating']]

# 2. Data Preprocessing
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if isinstance(text, str):  # Handle non-string values
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        tokens = word_tokenize(text)
        filtered_text = [w for w in tokens if not w in stop_words]
        return " ".join(filtered_text)
    else:
        return ""  # Return empty string if not a string

df['review'] = df['review'].apply(preprocess_text)

df.dropna(inplace=True)  # Drop rows with NaN reviews *after* preprocessing
df.reset_index(drop=True, inplace=True)  # Reset index after dropping rows!

# 3. Feature Extraction and Sentiment Conversion (COMBINED)
# Convert rating to sentiment (adapt to your data)
df['sentiment'] = df['rating'].apply(lambda x: 1 if x >= 4 else 0 if x <= 2 else 2) # Example: 4/5 stars = positive, 1/2 = negative, 3 = neutral

# Drop neutral reviews *before* creating X and y
df = df[df['sentiment'] != 2]

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['review'])  # Fit and transform *after* handling sentiment and dropping rows
y = df['sentiment']

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Missing values in X:", pd.DataFrame(X.toarray()).isnull().sum().sum())  # Check for missing values in X
print("Missing values in y:", y.isnull().sum())  # Check for missing values in y
print(y.value_counts()) # Check distribution of classes

# 4. Model Selection and Training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=2000)  # Increased max_iter
model.fit(X_train, y_train)

# 5. Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

# Model Saving
joblib.dump(model, 'amazon_sentiment_model.pkl')
joblib.dump(tfidf, 'amazon_tfidf_vectorizer.pkl')

# --- Manual Review Validation ---
manual_reviews = [
    "This product is amazing! It works perfectly and is exactly what I needed.",
    "I am very disappointed with this item. It arrived broken and is of poor quality.",
    "The delivery was fast, and the product was as described.",
    "The instructions were unclear, and the product is difficult to use.",
    "Great value for the price. I would definitely recommend it."
]
manual_labels = [1, 0, 1, 0, 1]  # Corresponding labels

preprocessed_manual = [preprocess_text(review) for review in manual_reviews]
manual_tfidf = tfidf.transform(preprocessed_manual)  # Use the *same* tfidf
manual_predictions = model.predict(manual_tfidf)

print("\n--- Manual Review Validation ---")
for i, review in enumerate(manual_reviews):
    print(f"Review: {review}")
    print(f"Predicted: {'Positive' if manual_predictions[i] == 1 else 'Negative'}")
    print(f"Actual: {'Positive' if manual_labels[i] == 1 else 'Negative'}")
    print("-" * 20)

# --- Insights and Challenges ---
# ... (your insights and challenges)

Please upload files :


Saving amazon_reviews.csv to amazon_reviews.csv
Shape of X: (905, 9199)
Shape of y: (905,)
Missing values in X: 0
Missing values in y: 0
sentiment
1    770
0    135
Name: count, dtype: int64
Accuracy: 0.8674033149171271
Precision: 0.8674033149171271
Recall: 1.0
F1-score: 0.9289940828402367

--- Manual Review Validation ---
Review: This product is amazing! It works perfectly and is exactly what I needed.
Predicted: Positive
Actual: Positive
--------------------
Review: I am very disappointed with this item. It arrived broken and is of poor quality.
Predicted: Positive
Actual: Negative
--------------------
Review: The delivery was fast, and the product was as described.
Predicted: Positive
Actual: Positive
--------------------
Review: The instructions were unclear, and the product is difficult to use.
Predicted: Positive
Actual: Negative
--------------------
Review: Great value for the price. I would definitely recommend it.
Predicted: Positive
Actual: Positive
--------------------
